In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mplsoccer import Pitch
import os
import ast

In [4]:
df=pd.read_csv(r"C:\Users\lahd2\OneDrive\Mehdaf\Ai_comp\JP1_events.csv")

C:\Users\lahd2\AppData\Local\Temp\ipykernel_14404\3890916516.py:1: DtypeWarning: Columns (94,95,98,108,111,113,119,120,121,123,124,125,126,127,128,129,130,131,132,133,135,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(r"C:\Users\lahd2\OneDrive\Mehdaf\Ai_comp\JP1_events.csv")


In [5]:
event=df
def extract_coordinates(coord_str):
    try:
        coords = ast.literal_eval(coord_str)
        if isinstance(coords, list) and len(coords) == 2:
            return coords[0], coords[1]
        else:
            return None, None
    except (ValueError, SyntaxError, TypeError):
        return None, None

def extract_3d_coordinates(coord_str):
    try:
        coords = ast.literal_eval(coord_str)
        if isinstance(coords, list) and len(coords) == 3:
            return coords[0], coords[1], coords[2]
        else:
            return None, None, None
    except (ValueError, SyntaxError, TypeError):
        return None, None, None

# Extract x, y from location
event['x'], event['y'] = zip(*event['location'].apply(extract_coordinates))

# Extract end_x, end_y from pass.end_location
event['end_x'], event['end_y'] = zip(*event['pass.end_location'].apply(extract_coordinates))

# Extract carry_end_x, carry_end_y from carry.end_location
event['carry_end_x'], event['carry_end_y'] = zip(*event['carry.end_location'].apply(extract_coordinates))

# Extract goalkeeper_end_x, goalkeeper_end_y from goalkeeper.end_location
event['goalkeeper_end_x'], event['goalkeeper_end_y'] = zip(*event['goalkeeper.end_location'].apply(extract_coordinates))

# Extract shot_end_x, shot_end_y, shot_end_z from shot.end_location
event['shot_end_x'], event['shot_end_y'], event['shot_end_z'] = zip(*event['shot.end_location'].apply(extract_3d_coordinates))

# Rename dotted column names to underscores
event.columns = [col.replace('.', '_') for col in event.columns]

# Drop unused original columns
event.drop(columns=[
    'location', 
    'pass_end_location', 
    'carry_end_location',
    'goalkeeper_end_location', 
    'shot_end_location',
    'id', 
    'related_events'
], inplace=True, errors='ignore')

# Display cleaned DataFrame
print(event)


         index  period     timestamp  minute  second  possession  duration  \
0            1       1  00:00:00.000       0       0           1    0.0000   
1            2       1  00:00:00.000       0       0           1    0.0000   
2            3       1  00:00:00.000       0       0           1    0.0000   
3            4       1  00:00:00.000       0       0           1    0.0000   
4            5       1  00:00:00.657       0       0           2    1.2832   
...        ...     ...           ...     ...     ...         ...       ...   
1244336   3691       2  00:52:37.392      97      37         199    2.4848   
1244337   3692       2  00:52:38.733      97      38         199    1.1447   
1244338   3693       2  00:52:39.877      97      39         199    0.9747   
1244339   3694       2  00:52:40.180      97      40         199    0.0000   
1244340   3695       2  00:52:40.180      97      40         199    0.0000   

         type_id    type_name  possession_team_id  ...     y  e

In [6]:
event['prog_pass'] = np.where((event['type_name'] == 'Pass'), 
                           np.sqrt((120 - event['x'])**2 + (40 - event['y'])**2) - np.sqrt((120 - event['end_x'])**2 + (40 - event['end_y'])**2), 0)
event['prog_carry'] = np.where((event['type_name'] == 'Carry'), 
                            np.sqrt((120 - event['x'])**2 + (40 - event['y'])**2) - np.sqrt((120 - event['carry_end_x'])**2 + (40 - event['carry_end_y'])**2), 0)
event['pass_or_carry_angle'] = np.degrees(np.arctan2(event['end_y'] - event['y'], event['end_x'] - event['x']))
event['pass_outcome_name'].fillna(1, inplace=True)

C:\Users\lahd2\AppData\Local\Temp\ipykernel_14404\3324011044.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  event['pass_outcome_name'].fillna(1, inplace=True)


In [7]:
events_df = event
output_dir = r"E:\Ai_com\match"
os.makedirs(output_dir, exist_ok=True)
# Group by match_id and save each group to its own CSV file
for match_id, match_df in events_df.groupby("match_id"):
    match_filename = f"{output_dir}/match_{str(match_id).zfill(4)}.csv"
    match_df.to_csv(match_filename, index=False)
print(f"✅ Done! All {events_df['match_id'].nunique()} match files saved to: {output_dir}")

✅ Done! All 376 match files saved to: E:\Ai_com\match


In [8]:
physical = pd.read_csv(r"C:\Users\lahd2\OneDrive\Mehdaf\Ai_comp\JP1_physical.csv")
player_map = pd.read_csv(r"C:\Users\lahd2\OneDrive\Mehdaf\Ai_comp\players_mapping.csv")
team_map = pd.read_csv(r"C:\Users\lahd2\OneDrive\Mehdaf\Ai_comp\teams_mapping.csv")
match_map = pd.read_csv(r"C:\Users\lahd2\OneDrive\Mehdaf\Ai_comp\matches_mapping.csv")

In [9]:
player_map.rename(columns={
    'wyscout_id': 'player_id_wyscout',
    'statsbomb_id': 'playerid'
}, inplace=True)

team_map.rename(columns={
    'wyscout_id': 'team_id_wyscout',
    'statsbomb_id': 'teamId'
}, inplace=True)

match_map.rename(columns={
    'wyscout_id': 'match_id_wyscout',
    'statsbomb_id': 'matchId'
}, inplace=True)
physical = pd.merge(physical, player_map, left_on='playerid', right_on='player_id_wyscout', how='left')
physical = pd.merge(physical, team_map, left_on='teamId', right_on='team_id_wyscout', how='left')
physical = pd.merge(physical, match_map, left_on='matchId', right_on='match_id_wyscout', how='left')
physical.drop(columns=[
    'player_id_wyscout', 'team_id_wyscout', 'match_id_wyscout'
], inplace=True)

In [10]:
physical.drop(columns=['matchId_x', 'teamId_x', 'playerid_x'], inplace=True)
physical.rename(columns={
    'matchId_y': 'match_id',
    'teamId_y': 'team_id',
    'playerid_y': 'player_id'
}, inplace=True)
cols = ['match_id', 'team_id', 'player_id'] + [col for col in physical.columns if col not in ['match_id', 'team_id', 'player_id']]
physical = physical[cols]
print(physical.head())


   match_id  team_id  player_id                                  label  \
0   3925378     1880    42145.0  Yokohama F. Marinos - Sagan Tosu, 0-1   
1   3925378     1880    42145.0  Yokohama F. Marinos - Sagan Tosu, 0-1   
2   3925378     1880    42145.0  Yokohama F. Marinos - Sagan Tosu, 0-1   
3   3925378     1880    42145.0  Yokohama F. Marinos - Sagan Tosu, 0-1   
4   3925378     1880    42145.0  Yokohama F. Marinos - Sagan Tosu, 0-1   

               dateutc             teamName    player  \
0  2024-07-03 10:00:00  Yokohama F. Marinos  J. Amano   
1  2024-07-03 10:00:00  Yokohama F. Marinos  J. Amano   
2  2024-07-03 10:00:00  Yokohama F. Marinos  J. Amano   
3  2024-07-03 10:00:00  Yokohama F. Marinos  J. Amano   
4  2024-07-03 10:00:00  Yokohama F. Marinos  J. Amano   

                      metric     phase  value  
0  Count Medium Acceleration   Session  146.0  
1  Count Medium Acceleration  1st Half  100.0  
2  Count Medium Acceleration  2nd Half   46.0  
3  Count Medium Acce

In [11]:
phase_time_ranges = {
    "1'-15'": (1, 15),
    "16'-30'": (16, 30),
    "31'-45+'": (31, 45),
    "46'-60'": (46, 60),
    "61'-75'": (61, 75),
    "76'-120+'": (76, 120)
}
target_metrics = [
    'Count Medium Acceleration', 'Count High Acceleration',
    'Count Medium Deceleration', 'Count High Deceleration',
    'Total Distance', 'M/min', 'Running Distance',
    'High Speed Running (HSR) Distance', 'Sprinting Distance',
    'High Intensity (HI) Distance', 'Count HSR', 'Count Sprint',
    'Count HI', 'Max Speed'
]
phys_pivot = physical.pivot_table(
    index=['match_id', 'team_id', 'player_id', 'phase'],
    columns='metric',
    values='value'
).reset_index()
phys_pivot.columns.name = None
phys_pivot.rename(columns=lambda x: x.strip().lower().replace(" ", "_").replace("(", "").replace(")", ""), inplace=True)
match_folder = r"E:\Ai_com\match"
output_folder = r"E:\Ai_com\match_enriched"
os.makedirs(output_folder, exist_ok=True)
for file in os.listdir(match_folder):
    if file.endswith(".csv"):
        match_path = os.path.join(match_folder, file)
        match_df = pd.read_csv(match_path)

        match_id = int(file.split("_")[1].split(".")[0])
        match_phys = phys_pivot[phys_pivot['match_id'] == match_id]

        enriched_rows = []

        for _, row in match_df.iterrows():
            minute = row.get('minute')
            if pd.isna(minute):
                enriched_rows.append(row.to_dict())
                continue

            # Determine the phase
            matching_phase = None
            for phase, (start, end) in phase_time_ranges.items():
                if start <= minute <= end:
                    matching_phase = phase
                    break

            if not matching_phase:
                enriched_rows.append(row.to_dict())
                continue

            # Find matching physical data
            match_phys_row = match_phys[
                (match_phys['team_id'] == row['team_id']) &
                (match_phys['player_id'] == row['player_id']) &
                (match_phys['phase'] == matching_phase)
            ]

            row_dict = row.to_dict()

            if not match_phys_row.empty:
                phys_values = match_phys_row.iloc[0].to_dict()
                row_dict.update(phys_values)

            enriched_rows.append(row_dict)

        enriched_df = pd.DataFrame(enriched_rows)
        enriched_df.to_csv(os.path.join(output_folder, file), index=False)
        print(f"✅ Enriched: {file}")


✅ Enriched: match_3925226.csv
✅ Enriched: match_3925227.csv
✅ Enriched: match_3925228.csv
✅ Enriched: match_3925229.csv
✅ Enriched: match_3925230.csv
✅ Enriched: match_3925231.csv
✅ Enriched: match_3925232.csv
✅ Enriched: match_3925233.csv
✅ Enriched: match_3925234.csv
✅ Enriched: match_3925235.csv
✅ Enriched: match_3925236.csv


C:\Users\lahd2\AppData\Local\Temp\ipykernel_14404\1243077751.py:30: DtypeWarning: Columns (16,45,50,56,57,58,77,79,81,85,86,87,88,91,94,96,98,99,101,102,103,105,106,108,110,111,112,116,118,119,120,122,123,125,126,130,131,133,135,140,141,142,144) have mixed types. Specify dtype option on import or set low_memory=False.
  match_df = pd.read_csv(match_path)


✅ Enriched: match_3925237.csv
✅ Enriched: match_3925238.csv
✅ Enriched: match_3925239.csv
✅ Enriched: match_3925240.csv
✅ Enriched: match_3925241.csv
✅ Enriched: match_3925242.csv
✅ Enriched: match_3925243.csv
✅ Enriched: match_3925244.csv
✅ Enriched: match_3925245.csv
✅ Enriched: match_3925246.csv
✅ Enriched: match_3925247.csv
✅ Enriched: match_3925248.csv
✅ Enriched: match_3925249.csv
✅ Enriched: match_3925250.csv
✅ Enriched: match_3925251.csv
✅ Enriched: match_3925252.csv
✅ Enriched: match_3925253.csv
✅ Enriched: match_3925254.csv
✅ Enriched: match_3925255.csv
✅ Enriched: match_3925256.csv
✅ Enriched: match_3925257.csv
✅ Enriched: match_3925258.csv
✅ Enriched: match_3925259.csv
✅ Enriched: match_3925260.csv
✅ Enriched: match_3925261.csv
✅ Enriched: match_3925262.csv
✅ Enriched: match_3925263.csv
✅ Enriched: match_3925264.csv
✅ Enriched: match_3925265.csv
✅ Enriched: match_3925266.csv
✅ Enriched: match_3925267.csv
✅ Enriched: match_3925268.csv
✅ Enriched: match_3925269.csv
✅ Enriched

C:\Users\lahd2\AppData\Local\Temp\ipykernel_14404\1243077751.py:30: DtypeWarning: Columns (16,17,45,49,53,55,56,57,64,66,68,70,72,74,75,77,79,81,83,84,85,86,87,88,90,91,94,96,98,99,101,102,103,104,106,108,110,111,116,117,118,119,120,121,122,123,124,125,131,134,136,137,138,139,141,142,143) have mixed types. Specify dtype option on import or set low_memory=False.
  match_df = pd.read_csv(match_path)


✅ Enriched: match_3925348.csv
✅ Enriched: match_3925349.csv
✅ Enriched: match_3925350.csv
✅ Enriched: match_3925351.csv
✅ Enriched: match_3925352.csv
✅ Enriched: match_3925353.csv
✅ Enriched: match_3925354.csv
✅ Enriched: match_3925355.csv
✅ Enriched: match_3925356.csv
✅ Enriched: match_3925357.csv
✅ Enriched: match_3925358.csv
✅ Enriched: match_3925359.csv
✅ Enriched: match_3925360.csv
✅ Enriched: match_3925361.csv
✅ Enriched: match_3925362.csv
✅ Enriched: match_3925363.csv
✅ Enriched: match_3925364.csv
✅ Enriched: match_3925365.csv
✅ Enriched: match_3925366.csv
✅ Enriched: match_3925367.csv
✅ Enriched: match_3925368.csv
✅ Enriched: match_3925369.csv
✅ Enriched: match_3925370.csv
✅ Enriched: match_3925371.csv
✅ Enriched: match_3925372.csv
✅ Enriched: match_3925373.csv
✅ Enriched: match_3925374.csv
✅ Enriched: match_3925375.csv
✅ Enriched: match_3925376.csv
✅ Enriched: match_3925377.csv
✅ Enriched: match_3925378.csv
✅ Enriched: match_3925379.csv
✅ Enriched: match_3925380.csv
✅ Enriched

C:\Users\lahd2\AppData\Local\Temp\ipykernel_14404\1243077751.py:30: DtypeWarning: Columns (16,17,50,64,66,68,70,72,74,77,79,81,84,86,88,91,94,96,99,101,103,106,112,116,117,118,119,120,122,123,124,125,130,131,134,135,138,139) have mixed types. Specify dtype option on import or set low_memory=False.
  match_df = pd.read_csv(match_path)


✅ Enriched: match_3925559.csv
✅ Enriched: match_3925560.csv
✅ Enriched: match_3925561.csv
✅ Enriched: match_3925562.csv
✅ Enriched: match_3925563.csv
✅ Enriched: match_3925564.csv
✅ Enriched: match_3925565.csv
✅ Enriched: match_3925566.csv
✅ Enriched: match_3925567.csv
✅ Enriched: match_3925568.csv
✅ Enriched: match_3925569.csv
✅ Enriched: match_3925570.csv
✅ Enriched: match_3925571.csv
✅ Enriched: match_3925572.csv
✅ Enriched: match_3925573.csv
✅ Enriched: match_3925574.csv
✅ Enriched: match_3925575.csv
✅ Enriched: match_3925576.csv
✅ Enriched: match_3925577.csv
✅ Enriched: match_3925578.csv
✅ Enriched: match_3925579.csv
✅ Enriched: match_3925580.csv
✅ Enriched: match_3925581.csv
✅ Enriched: match_3925582.csv
✅ Enriched: match_3925583.csv
✅ Enriched: match_3925584.csv
✅ Enriched: match_3925585.csv
✅ Enriched: match_3925586.csv
✅ Enriched: match_3925587.csv
✅ Enriched: match_3925588.csv
✅ Enriched: match_3925589.csv
✅ Enriched: match_3925590.csv
✅ Enriched: match_3925591.csv
✅ Enriched

C:\Users\lahd2\AppData\Local\Temp\ipykernel_14404\1243077751.py:30: DtypeWarning: Columns (16,50,52,53,55,75,77,79,83,90,91,96,98,99,101,102,104,106,108,110,111,112,116,118,119,120,122,124,125,130,131,132,133) have mixed types. Specify dtype option on import or set low_memory=False.
  match_df = pd.read_csv(match_path)


✅ Enriched: match_3925597.csv
✅ Enriched: match_3925598.csv
✅ Enriched: match_3925600.csv
✅ Enriched: match_3925601.csv
✅ Enriched: match_3925602.csv
✅ Enriched: match_3925603.csv
